In [1]:
import json
import os
from PIL import Image

# Đường dẫn
json_path = "/mnt/VLAI_data/ViVQA-X/ViVQA-X_val.json"
coco_img_dir = "/mnt/VLAI_data/COCO_Images/val2014/"

# Đọc file JSON
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

samples = []
for item in data:
    img_path = os.path.join(coco_img_dir, item["image_name"])
    # Mở ảnh thành PIL Image
    image = Image.open(img_path).convert("RGB")
    sample = {
        "question": item["question"],
        "image": image,  # <-- PIL Image object
        "image_path": img_path,
        "explanation": item["explanation"],  # list
        "answer": item["answer"],
        "question_id": item["question_id"]
    }
    samples.append(sample)

# Kiểm tra 1 sample
print(samples[0])

{'question': 'Đây có phải là bức ảnh chụp nhiều độ phơi sáng của vận động viên trượt tuyết mặc áo đen không?', 'image': <PIL.Image.Image image mode=RGB size=640x480 at 0x743840E31330>, 'image_path': '/mnt/VLAI_data/COCO_Images/val2014/COCO_val2014_000000393271.jpg', 'explanation': ['hình người mặc cùng một bộ quần áo khi trượt xuống dốc', 'có vẻ như là cùng một người trượt tuyết làm những pha nhào lộn khác nhau', 'cùng một người trượt tuyết xuất hiện nhiều lần'], 'answer': 'có', 'question_id': '393271001'}


In [2]:
SYSTEM_PROMPT_VIVQA_ENHANCED = (
    "<image>\nBạn là một trợ lý AI chuyên gia, có khả năng phân tích hình ảnh một cách cẩn thận và đa nghi. "
    "Nhiệm vụ của bạn là trả lời câu hỏi của người dùng dựa trên hình ảnh được cung cấp. "
    "Trước tiên, hãy thực hiện một chuỗi suy luận chi tiết bên trong cặp thẻ <thinking></thinking>. "
    "Sau khi hoàn tất quá trình suy luận, hãy cung cấp câu trả lời cuối cùng theo đúng định dạng yêu cầu.\n\n"
    "Câu hỏi: {question}\n\n"
    "ĐỊNH DẠNG BẮT BUỘC:\n"
    "<thinking>\n"
    "<SUMMARY>[Tóm tắt ngắn gọn về hình ảnh và yêu cầu của câu hỏi]</SUMMARY>\n"
    "<ANALYSIS>[Phân tích các chi tiết, vật thể, văn bản trong ảnh có liên quan trực tiếp đến câu hỏi. Liệt kê các bằng chứng quan sát được.]</ANALYSIS>\n"
    "<REASONING_STEPS>[Trình bày quá trình lập luận logic từng bước một. Từ các bằng chứng đã phân tích, làm thế nào để đi đến câu trả lời? Giải thích các mối liên hệ.]</REASONING_STEPS>\n"
    "<CONCLUSION>[Đưa ra kết luận cuối cùng từ quá trình lập luận trên.]</CONCLUSION>\n"
    "</thinking>\n"
    "<answer>[Điền câu trả lời trực tiếp và ngắn gọn vào đây]</answer>\n"
    "<explain>[Dựa vào quá trình suy luận trong <thinking>, giải thích cực kỳ ngắn gọn (trong khoảng 10-15 từ)]</explain>"
)

In [3]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch

model_id = "5CD-AI/Vintern-3B-R-beta"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=model_id,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
).eval().cuda()

/opt/miniconda3/envs/minhtq-vqanle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
/opt/miniconda3/envs/minhtq-vqanle/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


FlashAttention2 is not installed.


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  9.57it/s]


In [ ]:
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

# Constants
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    """Build image transformation pipeline"""
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    """Find the closest aspect ratio from target ratios"""
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
                
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    """Dynamically preprocess image into multiple tiles"""
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height
    
    # Calculate target ratios
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) 
        for i in range(1, n + 1) 
        for j in range(1, n + 1) 
        if i * j <= max_num and i * j >= min_num
    )
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])
    
    # Find closest aspect ratio
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size
    )
    
    # Calculate target dimensions
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]
    
    # Resize and split image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    
    # Add thumbnail if needed
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
        
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    """Load and preprocess image"""
    if isinstance(image_file, str):
        image = Image.open(image_file).convert('RGB')
    else:
        image = image_file.convert('RGB')
    
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    
    return pixel_values

# Generation config
generation_config = dict(
    max_new_tokens=1024, 
    do_sample=False, 
    num_beams=3, 
    repetition_penalty=2.5
)

# Process your sample
raw_image = samples[0]['image']
question = samples[0]['question']

# Load and preprocess image
pixel_values = load_image(raw_image, max_num=6).to(torch.bfloat16).cuda()

# Format question with image token
final_prompt = f"<image>\n{question}"




# 1) Lấy ID đúng của token ngữ cảnh ảnh
img_ctx_id = tokenizer.convert_tokens_to_ids("<IMG_CONTEXT>")
model.img_context_token_id = img_ctx_id
assert model.img_context_token_id is not None

# 2) Tính số token ảnh cần chèn
num_patches = pixel_values.shape[0]  # số “ô”/ảnh sau dynamic preprocess của bạn
num_img_tokens = model.num_image_token * num_patches  # thường là 256 * num_patches với cấu hình mặc định

# 3) Xây image span và prompt
image_span = "<img>" + "<IMG_CONTEXT>" * num_img_tokens + "</img>"
prompt = f"<|im_start|>user\n{image_span}\n{question}<|im_end|>\n<|im_start|>assistant\n"

# 4) Tokenize + cấu hình generate
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
eos_id = tokenizer.convert_tokens_to_ids("<|im_end|>")
gen_cfg = dict(
    max_new_tokens=512,
    do_sample=False,
    num_beams=3,
    repetition_penalty=1.1,
    eos_token_id=eos_id,
    pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
)


Assistant: Có, bức ảnh chụp nhiều độ phơi sáng của vận động viên trượt tuyết mặc áo đen. Điều này được thể hiện qua việc các vận động viên mặc áo khoác màu đen khác nhau, mỗi người có độ phơi sáng khác nhau, tạo nên sự đa dạng về màu sắc và độ tương phản trong bức ảnh.


In [ ]:
# # Generate response using custom chat method
# response, history = model.chat(
#     tokenizer, 
#     pixel_values, 
#     final_prompt, 
#     generation_config, 
#     history=None, 
#     return_history=True
# )

# print(f"User: {question}\nAssistant: {response}")

In [ ]:
# 5) Gọi generate (chú ý: KHÔNG cần image_flags; model tìm theo IMG_CONTEXT)
out = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    pixel_values=pixel_values,
    **gen_cfg
)

# 6) Decode
resp = tokenizer.decode(out[0], skip_special_tokens=True)
resp = resp.split("</s>")[0].split("<|im_end|>")[0].strip()  # gọn gàng lại nếu cần
print("Assistant:", resp)

In [ ]:
import torch

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
input_ids = inputs["input_ids"]
num_patches = pixel_values.shape[0]
image_flags = torch.ones((num_patches, 1), dtype=torch.long, device=model.device)
with torch.no_grad():
    out = model(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        pixel_values=pixel_values,    
        image_flags=image_flags,     
        use_cache=False,                  
        return_dict=True
    )

logits = out.logits  # shape: [batch, seq_len, vocab_size]

C                     p=0.4355
Đ                     p=0.3379
Không                 p=0.0588
D                     p=0.0315
B                     p=0.0278
